# Despliegue del endpoint de inferencia de SageMaker

Previo a la ejecución de este notebook, debe descomprimir el contenido del archivo [export.zip](https://github.com/DoradoReciclaAI/Project_recicla_ai/blob/main/rest_api/model/export.zip) dentro de este contenedor.

## Despliegue

Este paso desplegará el Modelo exportado y generará un EndPoint de inferencia para dicho modelo.

In [ ]:
import tarfile
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker import get_execution_role

In [ ]:
role = get_execution_role()

with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)
    
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

!touch train.py

sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '2.1',
                                  entry_point = 'train.py')

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---------------!CPU times: user 4.22 s, sys: 258 ms, total: 4.48 s
Wall time: 7min 37s


#### Copiar el nombre del end point creado
Cuáno finalice la creació del EndPoint, vaya en SageMaker a la sección correspondiente y copie el nombre de dicho EndPoint para poder invocarlo.

In [ ]:
endpoint_name = 'tensorflow-inference-2020-11-16-02-33-02-323'

## Probar el End Point Creado

Se debe subir la imágen de prueba correspondiente. En el ejemplo es images/O_12569.jpg pero puede ser cualquier otra imágen.

In [ ]:
import json
import boto3
import numpy as np
from PIL import Image

classes = ['cardboard', 'glass', 'metal', 'organic', 'paper', 'plastic', 'trash']
runtime= boto3.client('runtime.sagemaker')

img_name = 'images/O_12569.jpg'

data_s = Image.open(img_name)
image_res = data_s.resize((224,224), Image.BILINEAR)
image_dim = np.array(image_res)
image_dim = np.expand_dims(image_dim, axis=0)


payload = json.dumps(image_dim.tolist())

prediction = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='application/json',
                                   Body=payload)

results = json.loads(prediction['Body'].read().decode())['predictions'][0]

print('Class: ' + classes[np.argmax(results)] + ' ' + str(round(np.amax(results)*100, 2)) + ' %')

Class: organic 100.0 %
